# A Two-Hours Seminar about [InterCom](https://github.com/Tecnologias-multimedia/intercom)

## 1. What is InterCom
InterCom is a real-time intercommunicator (currently only transmit audio). The sequence of [frames](https://python-sounddevice.readthedocs.io/en/0.3.12/api.html) (two stereo samples) is splitted into chunks and each one is transmitted in an [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol) packet.

### A minimal implementation

In [ ]:
# Interrupt with: Kernel -> Interrupt
!python ../minimal.py --show_stats

## 2. Jitter and buffering
Non-dedicated links use to have a significant [jitter](https://en.wikipedia.org/wiki/Jitter).
To hide the it, we can use buffering:

![Buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/graphics/timelines.svg)

### Latency and jitter in my host

In [ ]:
!ping -c 10 localhost

Too ideal ...

### Let's increases latency and jitter
* Latency = 100ms
* Jitter = 100ms
* Correlation between RTTs = 0.25 (<1)
* Statistical distribution of the RTTs = normal

In [ ]:
# Remember to add "your_username_here ALL=(ALL) NOPASSWD: ALL" to the end of /etc/sudoers.
# Notice that these times express RTTs, not simple one-way lantencies.
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root handle 1: netem delay 100ms 100ms 25% distribution normal
!tc qdisc show dev lo

In [ ]:
!ping -c 10 localhost

### Let's listen again to minimal

In [ ]:
!python ../minimal.py --show_stats

Quite bad because the chunks don't arrive with a constant cadence to the receiver (see the previous figure).

### Let's listen to the version of InterCom that uses a buffer of 300 ms

In [ ]:
!python ../buffer.py -b 300 --show_stats

Much better!

### Remove tc rules
Optional while you are running this notebook.

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

## 3. Compression using [DEFLATE](https://en.wikipedia.org/wiki/DEFLATE)
We have "hidden" the jitter, but ... what happens if the available transmission bandwidth ([throughput](https://en.wikipedia.org/wiki/Throughput)) is not ehougn?

Compression can helo to reduce the bit-rate. The chunks are compressed using [LZSS](https://en.wikipedia.org/wiki/Lempel-Ziv-Storer-Szymanski) (that is based on [LZ77](https://github.com/vicente-gonzalez-ruiz/LZ77)) and [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding) (see also [this](https://vicente-gonzalez-ruiz.github.io/Huffman_coding/)). This text compression algorithm is used because:
1. Works well when repeated strings are found at the input.
2. I's available in [The Standard Python Library](https://docs.python.org/3/library/) ([zlib](https://docs.python.org/3/library/zlib.html)).

### A throughput computation
Notice that this measurement depends heavely on the packet size, that using ping is limited to 64 KB. For this reason, the results can be only approximated :-/

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo
!ping -c 1 -s 65527 localhost # Header length: 9 bytes

In [ ]:
!printf "Gbps = "
!echo 65527*8/0.147/2/1000/1000 | bc -l

### Limit the transmission bit-rate to 200 kbps
And also set the previous jitter.

In [ ]:
!sudo tc qdisc add dev lo root handle 1: netem delay 100ms 100ms 25% distribution normal
!sudo tc qdisc add dev lo parent 1:1 handle 10: tbf rate 200kbit burst 1024kbit limit 1024kbit

In [ ]:
!ping -c 1 -s 65527 localhost

In [ ]:
!printf "kbps = "
!echo 65527*8/359/2 | bc -l

### Let's try again

In [ ]:
!python ../buffer.py -b 300 --show_stats

Notice that some chunks are lost. DEFLATE may not be enought!

### Remove tc rules

In [ ]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

## 4. Bit-rate "control" through [quantization](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb)

Quantization removes the less relevant information (mainly [electronic noise](https://en.wikipedia.org/wiki/Noise_(electronics)) ...) and helps to increase the [compression ratio](https://en.wikipedia.org/wiki/Compression_ratio). In lossy signal compression, [dead-zone quantizers](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb) are commonly used.

In InterCom, we "control" the bit-rate in a lossely way because real-time bit-rate control through the quantization step $\Delta$ is computationally intensive (we must determine the [Rate/Distortion curve](https://en.wikipedia.org/wiki/Rate%E2%80%93distortion_theory) of the current chunk to find $\Delta$ before to quantize, compress and send it). The current implementation estimates the number of lost chunks per second and use this information to increase or decrease the quantization step for the chunks of the next second.

### Adapting to the current throughput

In [ ]:
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root tbf rate 200kbit burst 1024kbit limit 1024kbit
!tc qdisc show dev lo
!python ../br_control.py -b 300 --show_stats

## 5. Spatial (inter-channel) decorrelation
The samples of a (stero) frame tend to have similar amplitudes. For this reason, we apply [Mid/Side coding](https://tecnologias-multimedia.github.io/study_guide/spatial_decorrelation/) (before quantization).

## 6. Temporal (intra-channel) decorrelation
The samples of a channel exhibit temporal redundancy. Therefore, we use a [Discrete Wavelet Transform (DWT)](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/) to exploit it (before quantization).

## 7. Work to do
1. Currently, quantization minimizes the MSE (Mean Square Error). Perceptual quantization can increase the compression ratio without increasing the perceived distortion.
2. Transmit video.